# 문장 임베딩
- BERT
- OpenAI

## BERT sentence transformers 설치
- uv pip install sentence_transformers ipywidgets

In [2]:
from sentence_transformers import SentenceTransformer

In [5]:
import pandas as pd

# import corpus
data = pd.read_table('ratings_train.txt')[:10]
data

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1


In [4]:
# BERT 모델 호출
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [16]:
data['embedding'] = data['document'].apply(lambda x: model.encode(x))
data

,id,document,label,embedding
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0,"[0.1199709, 0.036388524, 0.9433599, -0.0292631..."
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1,"[0.056858446, 0.020919, 0.7253657, 0.05481379,..."
2,10265843,너무재밓었다그래서보는것을추천한다,0,"[0.07181791, -0.018124584, 0.84919286, 0.05228..."
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0,"[0.042829074, 0.025457421, 0.8471478, 0.046056..."
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1,"[-0.34263313, 0.22370131, 0.23183072, 0.021585..."
5,5403919,막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.,0,"[-0.1284231, -0.04270424, 0.5165525, -0.005466..."
6,7797314,원작의 긴장감을 제대로 살려내지못했다.,0,"[0.3519894, -0.45238355, 0.65755564, 0.2169624..."
7,9443947,별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단...,0,"[-0.3850559, -0.101053715, 0.76362723, -0.2009..."
8,7156791,액션이 없는데도 재미 있는 몇안되는 영화,1,"[0.23668765, -0.029815, 0.70949787, 0.21230657..."
9,5912145,왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나?,1,"[0.082663335, -0.04860803, 0.5354299, 0.112480..."


In [25]:
import numpy as np

embeddings = np.array(data['embedding'].tolist())
embeddings.shape

(10, 768)

## OpenAI API
- OpenAI 가입
- API 이용으로 로그인
- API 첫 이용자 무료 크레딧 제공
- OpenAI API key 발급 및 저장

## OpenAI API를 활용한 임베딩
- openai 라이브러리 설치(uv pip install --upgrade openai)

In [28]:
from openai import OpenAI

In [56]:
openai_client = OpenAI()
openai_embedding_model = 'text-embedding-3-small'

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [46]:
def generate_openai_embedding(text):
    embedding = openai_client.embeddings.create(
        model = openai_embedding_model,
        input=text
    )
    
    return embedding.data[0].embedding

In [47]:
for text in data['document']:
    print(text)
    embeddings = generate_openai_embedding(text)
    embeddings = np.array(embeddings)
    print(embeddings, embeddings.shape)
    break

아 더빙.. 진짜 짜증나네요 목소리
[-0.00100734  0.02021617 -0.08748494 ... -0.00089626  0.01053018
 -0.0231042 ] (1536,)


# OpenAI를 활용
- ChatGPT prompt

In [53]:
def create_openai_message(prompt, system_message):
    return [
        {'role':'system', 'content':system_message},
        {'role':'user', 'content':prompt}
    ]

def get_openai_response(prompt, system_message):
    messages = create_openai_message(prompt, system_message)
    completion = openai_client.chat.completions.create(
        model = 'gpt-3.5-turbo-0125',
        messages = messages
    )
    return completion.choices[0].message.content

In [50]:
system_message = """ You will be working as an AI text editor specializing in Korean. 
You will be reviewing the movie reviews in Korean. Rephrase the review using standard Korean language and correct any typos.
Your response will  be from a first-person perspective, ensuring there is not grammar error or typo.
stay focused on the content while rephrasing the review, and avoid overestimating what the text is originally meant to convey.
Before suggesting an answer, make sure to use standard language and ensure there is no misspelled word.
Do no answer in English, but only in Korean.
"""

In [55]:
for text in data['document']:
    response = get_openai_response(text, system_message)
    print(text, '>>', response)
    break

아 더빙.. 진짜 짜증나네요 목소리 >> 더빙이 정말 짜증나요.
